# Type Filtering Pipeline Example

This notebook implements a [Gotaglio](https://github.com/MikeHopcroft/gotaglio) pipeline that uses
`ts-type-filter` to optimize LLM prompts for a ficticious restaurant ordering bot. The user can can ask the bot for items from the [menu](./menu.ts) and the bot will update a shopping cart. The conversation can continue through multiple turns, each of which modifies the cart.

Before we get started this is one small Jupyter detail relating to the fact that
[Gotaglio](https://github.com/MikeHopcroft/gotaglio) library uses
[asyncio](https://docs.python.org/3/library/asyncio.html).
Since Jupyter notebooks run their own event loops, it is necessary to use a package like [nest_asyncio](https://pypi.org/project/nest-asyncio/) to allow `gotaglio` to run async operations inside the notebook's event loop. Just include this code somewhere in your notebook. [NOTE: perhaps it would make sense to incorporate this directly into the Gotaglio library.]

In [1]:
import nest_asyncio

# Necessary for running asyncio in Jupyter
nest_asyncio.apply()

In this case we'll be using a using the `MenuPipeline` class defined in [ordering.py](./ordering.py).
We'll create an instance of `Gotaglio` that knows about this pipeline.
* The first parameter is the list of pipeline classes.
* The second parameter overrides the default Gotaglio configuration. I intend to put my model configuration (models.json) and credentials (.credentials.json) files at the root of this repo. Since this notebook file is two levels down in samples/menu, I set the `base_folder` property to `../..`. Alternatively I could have put the configuration files in the same folder as the notebook, but I want to be able to share them across all samples.


In [ ]:
from gotaglio.gotag import Gotaglio, read_json_file
import samples.menu.ordering as sample

gt = Gotaglio([sample.MenuPipeline], {"base_folder": "../.."})

We'd like to run `MenuPipeline` on the cases in [data/multi_turn_cases.json](./data/multi_turn_cases.json). These are multi-turn test cases that have an initial shopping `cart` prooperty and an array of `turns`, each of which has a user `query` and an `expected` cart. Each test case also has a `uuid` and an optional `keywords` array.

Although we specify multi-turn cases, we would like to analyze each turn, independent of the turns that come before. This allows us to collect information about the correctness of each turn on its own and combine that information to assess complete sequences of turns.

We use a function called `flatten_cases` to construct a new set of single-turn cases, based on the original multi-turn cases. The single-turn cases have uuids based on the orginal uuid and the turn number (basically uuid.turn). The initial `cart` for each turn is just the `expected` cart from the previous turn (or the initial cart in the case of the first turn).

In the following code, we read `multi_turn_cases.json` and flatten it.

Then we use `gt.run()` to run the flattened cases:
* The first parameter specifies the name of the pipeline we want to run. We specify a name because `Gotaglio` can be instantiated with multiple pipelines. The pipeline name is specified in the `_name` static member of the class implementing the pipeline.
* The second parameter provides the test cases. You can either pass the path to a json file with the test cases, or you can pass a dictionary. In this case we pass a dictionary because we want to use the result of `flatten_cases()`.
* The third parameter is a dictionary of [glom](https://glom.readthedocs.io/en/latest/)-paths to pipeline configuration value overrides. We specify [template.txt](./template.txt) as a [jinja2](https://pypi.org/project/Jinja2/) template for the LLM prompt. We specify the built-in `perfect` model that always gives the correct answer.
* The `save` parameter indicates that the run log should also be written to the default log location. Run logs are useful for analysis and as a starting point for new runs with some configuration changes.

Finally, we use `gt.format()` to display a richly formatted transcript of each case.

In [4]:
import samples.menu.flatten_cases

# Load and flatten the cases from the JSON file
cases = samples.menu.flatten_cases(read_json_file("data/multi_turn_cases.json"))

# Run the pipeline on the flattened cases.
runlog1 = gt.run(
    "menu",
    cases,
    {
        "prepare.template": "template.txt",
        "infer.model.name": "gpt4o",
    },
    save=True,
)

# Format the results.
gt.format(runlog1)

  Summary for e1e3455d-bb94-492d-889e-2182f58f5817  
┏━━━━━━━┳━━━━━━━━━━┳━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃    id ┃ run      ┃ score ┃ keywords              ┃
┡━━━━━━━╇━━━━━━━━━━╇━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0e6.0 │ COMPLETE │  0.00 │ meal, p0, single-turn │
│ c47.0 │ COMPLETE │  1.00 │ combo, multi-turn, p1 │
│ c47.1 │ COMPLETE │  1.00 │ combo, multi-turn, p1 │
│ 894.0 │ COMPLETE │  0.00 │ edit, p1, single-turn │
│ adf.0 │ COMPLETE │  0.00 │ edit, multi-turn, p2  │
│ adf.1 │ COMPLETE │  0.00 │ edit, multi-turn, p2  │
│ 7ee.0 │ COMPLETE │  0.00 │                       │
└───────┴──────────┴───────┴───────────────────────┘

Total: 7
Complete: 7/7 (100.00%)
Error: 0/7 (0.00%)
Passed: 5/7 (71.43%)
Failed: 2/7 (28.57%)


Results written to ../../logs\e1e3455d-bb94-492d-889e-2182f58f5817.json


## Run: e1e3455d-bb94-492d-889e-2182f58f5817
## Case: 0e6.0
**PASSED**  

Keywords: single-turn, meal, p0  
Complete menu tokens: 1296  
Input tokens: 517, output tokens: 71

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=WiseguyMeal<ComboSizes>|Meal<ComboSizes>|Wiseguy|ComboTwo|ComboThree|FountainDrink<any,any>;
type WiseguyMeal<SIZE extends ComboSizes>={name:"Wiseguy 
Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink<SIZE>};
type ComboSizes="Large"|CHOOSE;
type CHOOSE="CHOOSE";
type Wiseguy=GenericWiseguy<"Wiseguy"|"Vegan Wiseguy"|"Double Wiseguy"|"Triple Wiseguy"|"Down East Wiseguy">;
type GenericWiseguy<NAME>={name:NAME,type:"Regular"|"With Bacon"|"With Cheese"|"With Bacon and Cheese"|CHOOSE};
type ChooseDrink<SIZE extends DrinkSizes>=FountainDrink<any,SIZE>|CHOOSE;
type DrinkSizes="Large"|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type Meal<SIZE extends 
ComboSizes>={name:"Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink<SIZE>};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=Wiseguy|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _a large wiseguy meal with coke_

**assistant:**
```json
{
  "items": [
    {
      "name": "Wiseguy Meal",
      "size": "Large",
      "sandwich": "CHOOSE",
      "fries": "CHOOSE",
      "drink": {
        "name": "Coca-Cola",
        "size": "Large"
      }
    }
  ]
}
```

**No repairs**

**Full query**:
* a large wiseguy meal with coke

## Case: c47.0
**FAILED:** cost=1  

Keywords: multi-turn, combo, p1  
Complete menu tokens: 1296  
Input tokens: 249, output tokens: 46

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=Burger|ComboTwo;
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericBurger<"Bacon Cheeseburger">|CHOOSE;
type CHOOSE="CHOOSE";
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _can I get a cheeseburger twofer_

**assistant:**
```json
{
  "items": [
    {
      "name": "Twofer Combo",
      "one": {
        "name": "Cheeseburger"
      },
      "two": "CHOOSE"
    }
  ]
}
```

**Repairs:**
* 0: Twofer Combo: change one to `{'name': 'Bacon Cheeseburger'}`

**Full query**:
* can I get a cheeseburger twofer

## Case: c47.1
**FAILED:** cost=1  

Keywords: multi-turn, combo, p1  
Complete menu tokens: 1296  
Input tokens: 503, output tokens: 87

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Burger|KoreanChicken|ComboTwo|ComboThree|FountainDrink<any,any>;
type PattyMelt={name:"Bacon Melt"|"Mushroom and Swiss Melt",options?:(Bacon|Condiments)[]};
type Bacon={amount:Optional,name:"Bacon"};
type Optional="Regular";
type Condiments={amount:Amount,name:"Ketchup"};
type Amount="Regular"|"Extra";
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME,options?:(Bacon|Condiments)[]};
type KoreanChicken={name:"Sweet and Spicy Chicken",options?:(Bacon|Condiments)[]};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericBurger<"Bacon Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. 
Pepper"|"Sprite","Medium">|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Sprite"|"Sprite Zero";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": [
    {
      "name": "Twofer Combo",
      "one": {
        "name": "Bacon Cheeseburger"
      },
      "two": "CHOOSE"
    }
  ]
}
```

**user:** _can I get that with extra ketchup and a sprite_

**assistant:**
```json
{
  "items": [
    {
      "name": "Twofer Combo",
      "one": {
        "name": "Bacon Cheeseburger",
        "options": [
          {
            "amount": "Extra",
            "name": "Ketchup"
          }
        ]
      },
      "two": {
        "name": "Sprite",
        "size": "CHOOSE"
      }
    }
  ]
}
```

**Repairs:**
* 0: Twofer Combo: change two to `{'name': 'Sprite', 'size': 'Medium'}`

**Full query**:
* can I get that with extra ketchup and a sprite
* CHOOSE
* Twofer Combo
* Bacon Cheeseburger

## Case: 894.0
**PASSED**  

Keywords: single-turn, edit, p1  
Complete menu tokens: 1296  
Input tokens: 434, output tokens: 79

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|FountainDrink<any,any>;
type PattyMelt={name:"Mushroom and Swiss Melt"};
type Chicken=GenericChicken<"Grilled Chicken Sandwich"|"Cordon Bleu">;
type GenericChicken<NAME>={name:NAME};
type KoreanChicken={name:"Sweet and Spicy Chicken"|"Seasame Soy Chicken"|"Spicy Garlic Chicken"};
type Pitas={name:"Lemon Chicken Pita"|"Smokey Chicken Pita"|"Tangy Chicken Pita"};
type Fish={name:"Captain Nemo Burger"};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon 
Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type GenericBurger<NAME>={name:NAME};
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _i want a a grilled chicken and a fish burger and two cokes_

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich"
    },
    {
      "name": "Captain Nemo Burger"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**No repairs**

**Full query**:
* i want a a grilled chicken and a fish burger and two cokes

## Case: adf.0
**PASSED**  

Keywords: multi-turn, edit, p2  
Complete menu tokens: 1296  
Input tokens: 434, output tokens: 79

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|FountainDrink<any,any>;
type PattyMelt={name:"Mushroom and Swiss Melt"};
type Chicken=GenericChicken<"Grilled Chicken Sandwich"|"Cordon Bleu">;
type GenericChicken<NAME>={name:NAME};
type KoreanChicken={name:"Sweet and Spicy Chicken"|"Seasame Soy Chicken"|"Spicy Garlic Chicken"};
type Pitas={name:"Lemon Chicken Pita"|"Smokey Chicken Pita"|"Tangy Chicken Pita"};
type Fish={name:"Captain Nemo Burger"};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon 
Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type GenericBurger<NAME>={name:NAME};
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Diet Coke"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _i want a a grilled chicken and a fish burger and two cokes_

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich"
    },
    {
      "name": "Captain Nemo Burger"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**No repairs**

**Full query**:
* i want a a grilled chicken and a fish burger and two cokes

## Case: adf.1
**PASSED**  

Keywords: multi-turn, edit, p2  
Complete menu tokens: 1296  
Input tokens: 669, output tokens: 110

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=PattyMelt|Burger|Chicken|KoreanChicken|Pitas|Fish|ComboTwo|ComboThree|FountainDrink<any,any>;
type PattyMelt={name:"Bacon Melt"|"Mushroom and Swiss Melt",options?:(Bacon|Condiments)[]};
type Bacon={amount:Optional,name:"Bacon"};
type Optional="Regular";
type Condiments={amount:Amount,name:"Mayo"};
type Amount="Regular"|"Extra";
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME,options?:(Bacon|Condiments)[]};
type Chicken=GenericChicken<"Grilled Chicken Sandwich"|"Cordon Bleu">;
type GenericChicken<NAME>={name:NAME,options?:(Bacon|Condiments)[]};
type KoreanChicken={name:"Sweet and Spicy Chicken"|"Seasame Soy Chicken"|"Spicy Garlic 
Chicken",options?:(Bacon|Condiments)[]};
type Pitas={name:"Lemon Chicken Pita"|"Smokey Chicken Pita"|"Tangy Chicken Pita",options?:(Bacon|Condiments)[]};
type Fish={name:"Captain Nemo Burger",options?:(Bacon|Condiments)[]};
type ComboTwo={name:"Twofer Combo",one:TwoThreeChoices,two:TwoThreeChoices};
type TwoThreeChoices=GenericChicken<"Grilled Chicken Sandwich">|GenericBurger<"Bacon 
Cheeseburger">|FountainDrink<"Coca-Cola"|"Diet Coke"|"Dr. Pepper"|"Sprite","Medium">|CHOOSE;
type FountainDrink<NAME extends DrinkNames,SIZE extends DrinkSizes>={name:NAME,size:SIZE};
type DrinkNames="Coca-Cola"|"Coca-Cola Zero Sugar";
type DrinkSizes="CHOOSE";
type CHOOSE="CHOOSE";
type ComboThree={name:"Threefer Combo",one:TwoThreeChoices,two:TwoThreeChoices,three:TwoThreeChoices};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich"
    },
    {
      "name": "Captain Nemo Burger"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**user:** _add bacon and extra mayo the chicken and 86 the fish_

**assistant:**
```json
{
  "items": [
    {
      "name": "Grilled Chicken Sandwich",
      "options": [
        {
          "amount": "Regular",
          "name": "Bacon"
        },
        {
          "amount": "Extra",
          "name": "Mayo"
        }
      ]
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    },
    {
      "name": "Coca-Cola",
      "size": "CHOOSE"
    }
  ]
}
```

**No repairs**

**Full query**:
* add bacon and extra mayo the chicken and 86 the fish
* Grilled Chicken Sandwich
* Coca-Cola
* CHOOSE
* Captain Nemo Burger

## Case: 7ee.0
**PASSED**  

Keywords:   
Complete menu tokens: 1296  
Input tokens: 490, output tokens: 76

**system:**
```json
Your job is to convert text-based restaurant orders into JSON data structures.
You never return a text answer. You always generate a JSON data structure.
The JSON should conform to the following typescript type definition for type `Cart`:

~~~typescript
type Cart={items:Item[]};
type Item=WiseguyMeal<ComboSizes>|Wiseguy|PattyMelt|Burger|KoreanChicken;
type WiseguyMeal<SIZE extends ComboSizes>={name:"Wiseguy 
Meal",size:SIZE,sandwich:Wiseguy|CHOOSE,fries:CHOOSE,drink:ChooseDrink<SIZE>};
type ComboSizes="CHOOSE";
type Wiseguy=GenericWiseguy<"Wiseguy"|"Vegan Wiseguy"|"Double Wiseguy"|"Triple Wiseguy"|"Down East Wiseguy">;
type GenericWiseguy<NAME>={name:NAME,type:"Regular"|"With Bacon"|"With Cheese"|"With Bacon and 
Cheese"|CHOOSE,options?:Veggies};
type CHOOSE="CHOOSE";
type Veggies={amount:ExtraAmount,name:"Tomato"};
type ExtraAmount="No"|"Regular"|"extra";
type ChooseDrink<SIZE extends DrinkSizes>=CHOOSE;
type DrinkSizes="CHOOSE";
type PattyMelt={name:"Mushroom and Swiss Melt",options?:(Veggies|Condiments)[]};
type Condiments={amount:Amount,name:"Mayo"};
type Amount="No"|"Regular"|"Extra";
type Burger=GenericBurger<"Bacon Double Cheeseburger"|"Bacon Cheeseburger"|"Double Cheeseburger"|"Cheeseburger">;
type GenericBurger<NAME>={name:NAME,options?:(Veggies|Condiments)[]};
type KoreanChicken={name:"Sweet and Spicy Chicken",options?:(Veggies|Condiments)[]};
~~~

* Only use CHOOSE when the user didn't specify a choice and there is no template literal contraining the choice.
* If request doesn't match the schema exactly, choose the closest matching item that is strictly legal for the 
schema.
* Combos that don't specify enough items should use CHOOSE instead of duplicating an item.
* Return the JSON object of type `Cart`.
```

**assistant:**
```json
{
  "items": []
}
```

**user:** _can I a double wiseguy with no tomatoes and extra mayo_

**assistant:**
```json
{
  "items": [
    {
      "name": "Double Wiseguy",
      "type": "CHOOSE",
      "options": [
        {
          "amount": "No",
          "name": "Tomato"
        },
        {
          "amount": "Extra",
          "name": "Mayo"
        }
      ]
    }
  ]
}
```

**No repairs**

**Full query**:
* can I a double wiseguy with no tomatoes and extra mayo
